In [1]:
import os 
import openai 
from dotenv import load_dotenv, find_dotenv
import json

dotenv_path = os.path.join('..', '.env')
load_dotenv(dotenv_path)
openai.api_key = os.environ['OPENAI_API_KEY']

#### Tutorials of using langchain for extraction etc.

https://learn.deeplearning.ai/courses/functions-tools-agents-langchain/lesson/5/tagging-and-extraction

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()
chain = prompt | model | output_parser
chain.invoke({"topic": "ok"})

'Why did the "OK" sign break up with the thumbs up? \n\nBecause it found someone more "a-peeling"!'

#### Tagging

* tagging function
* Given a function description, select arguments from the input text genreate a structure output forming a function call
* more generally - LLM can evaluate the input text and generate a structured output

In [4]:


from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

convert_pydantic_to_openai_function(Tagging)
model = ChatOpenAI(model="gpt-4o-mini",temperature=0.1)
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

tagging_chain = prompt | model_with_functions
tagging_chain.invoke({"input": "I love langchain"})

/journel/s0/zur74/venv/gpt_rag/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"en"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 105, 'total_tokens': 115}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b0abffe81', 'finish_reason': 'stop', 'logprobs': None}, id='run-b23d7bf4-d974-498d-a66e-b4e9ac49bd60-0', usage_metadata={'input_tokens': 105, 'output_tokens': 10, 'total_tokens': 115})

#### Extraction

* when given an input json schema, the LLM has been fine tuned to find and  fill in the parameters of that schema
* can be used as general purpose extraction.

In [5]:
from typing import Optional
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")


class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

# you can add more functions here
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(
                functions=extraction_functions,
                function_call={"name": "Information"}
                )

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])


extraction_chain = prompt | extraction_model

extraction_chain.invoke(
    "Joe is 30, his mom is Martha. She is 60. Tom is 40 and his daughter is 10."
    )


# add json parser
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()
a = extraction_chain.invoke(
    "Joe is 30, his mom is Martha. She is 60. Tom is 40 and his daughter is 10."
    )
print(a)

#### Embedding

* embedding text into a series of chunks


In [6]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls = ["https://lilianweng.github.io/posts/2023-06-23-agent/"]

loader =  UnstructuredURLLoader(urls=urls)
documents = loader.load()
documents = documents[0]

class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()
